In [38]:
using HDF5, EzXML, Glob, Dates, Plots, LineSearches, LineSearches, JuMP, StatsBase, BayesOpt, DataFrames, CSV
import NLopt
include("/Users/jjc/CSF/CSF Dynamics Scripts/readCSF.jl")

# datapath = "/Users/jjc/Documents/SSC/recs_june_2022"
datapath = "/Users/jjc/Documents/SSC/CSF Data/selected_recordings"
path = pwd();
savepath = "/Users/jjc/CSF/"
files = glob("*.hdf5", datapath);

df = DataFrame(filename=files, Rcsf=0.1, E=0.1, Ib=0.1, P0=0.1, mode="NaN", error=0.0, errorICM=0.0)

# for f = 1:length(files)
for f = 2:2
    filename = files[f]
    global Data = readCSF(filename) # Read all data from the hdf5 file
    # global fitErrorVal = 0

    try
        global Pm = Data["ICP"][Data["infusion_start_frame"]:Data["infusion_end_frame"]]
    catch
        continue
    end

    lowerbound = [0.01, 0.001, -10.0] # Lower boundary
    upperbound = [50.0, 1.0, Data["P_b"]] # Upper boundary
    global Ib_lower = 0.01 # I_b range 
    global Ib_upper = 1.0 # I_b range
    global κ = 2 # penalty multiplier
    global C = 10^3 # penalty multiplier

    # BayesOpt params
    bkernel = "kMaternISO5"
    bsctype = SC_MAP
    bltype = L_MCMC

    # NLOpt params
    optalg = :GN_DIRECT_L

    # global Rcsf, E, P_0 = getModelBayes(lowerbound, upperbound, bkernel, bsctype, bltype)
    global Rcsf, E, P_0 = getModelNL(lowerbound, upperbound, optalg)
    I_b = (Data["P_b"] - P_0) / Rcsf

    plot_model(I_b, E, P_0, Data["ICP"], 20, false)
    # title!("I_b = $I_b\n" * "Rcsf = $(value(Rcsf))\n" * "E = $(value(E))\n" * "P_0 = $(value(P_0)))\n" * "error = $fitErrorVal")
    savefig("/Users/jjc/Documents/SSC/recs_june_2022/new/" * filename[length(datapath)+2:end-5] * "_New.pdf")

    global df.Rcsf[f] = round(Rcsf, digits=3)
    global df.P0[f] = round(P_0, digits=3)
    global df.Ib[f] = round(I_b, digits=3)
    global df.E[f] = round(E, digits=3)
    global df.filename[f] = filename[length(datapath)+2:end]
    global df.error[f] = round(fitErrorVal,digits=3)
    # global df.errorICM[f] = Data["fitGoodness"]
    # Data["mode"] == "dynamic" ? mode = "dynamic" : mode = "static"
    # global df.mode[f] = mode
    # CSV.write("/Users/jjc/Documents/SSC/recs_june_2022/results_raw_new.csv", df);
end

In [8]:
Data

Dict{String, Any} with 19 entries:
  "end_time"             => DateTime("2013-05-30T15:08:21")
  "P_0"                  => 6.62
  "T"                    => [0.0, 0.166667, 0.333333, 0.5, 0.666667, 0.833333, …
  "Rcsf"                 => 14.29
  "P_b"                  => 13.03
  "AMP"                  => [NaN, 0.250628, 0.191833, 0.285034, 0.282389, 0.665…
  "start_time"           => DateTime("2013-05-30T14:22:56")
  "I_inf"                => 1.5
  "one_needle"           => 1
  "I_b"                  => 0.45
  "infusion_start_frame" => 70
  "plateau_start"        => 178
  "plateau_end"          => 204
  "E"                    => 0.11
  "rec_dur_s"            => 2725.0
  "ICP"                  => [13.9012, 13.4404, 14.202, 14.3033, 13.3949, 14.384…
  "P_p"                  => 33.23
  "infusion_end_frame"   => 203
  "Rn"                   => 1.89

In [34]:
using HypothesisTests
dfBayes = CSV.File("/Users/jjc/Documents/SSC/recs_june_2022/results_raw_newBayes.csv")
dfNLOpt = CSV.File("/Users/jjc/Documents/SSC/recs_june_2022/results_raw_new.csv")
histogram(dfBayes["error"])
histogram!(dfNLOpt["error"], alpha=0.5)

println(mean(dfBayes["error"]))
println(mean(dfNLOpt["error"]))

pvalue(OneSampleTTest(vec(dfBayes["error"]), vec(dfNLOpt["error"])))



2.3752083333333336
2.168625


1.4752038985239987e-8